0

In [20]:
#
#
# spit and embed by chunking
#
#
import re
import uuid
import chromadb

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings

persistent_client = chromadb.PersistentClient()

collection_name = 'all-MiniLM-L6-v2_1000_split_clean'

try:
    persistent_client.delete_collection(name=collection_name)
except:
    print('nothing to delete')

collection = persistent_client.get_or_create_collection(collection_name)

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

pdfs = ['London_Borough_of_Southwark',
        'London_Borough_of_Tower_Hamlets', 'London_Borough_of_Islington']

for pdf in pdfs[0:3]:
    with open('txts/' + pdf + '.txt') as f:
        string = f.read()
        cleaner_string = string.replace('\n', ' ').replace('\r', '')

        clean_string = re.sub("\s\s+", " ", cleaner_string)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            length_function=len,
            is_separator_regex=False,
        )

        # Split text
        split_texts = text_splitter.create_documents([clean_string])
        split_texts_list = [str(txt.page_content) for txt in split_texts]
        display(len(split_texts_list))

        # Embed text
        embedded_texts = embedding_model.embed_documents(
            texts=split_texts_list)

        # add vectors to collection
        ids = [str(uuid.uuid4()) for sent in split_texts_list]
        metadatas = [{"LPA": pdf}
                     for sent in split_texts_list]
        collection.add(
            embeddings=embedded_texts,
            documents=split_texts_list,
            ids=ids,
            metadatas=metadatas
        )

nothing to delete


963

849

1021

In [15]:
#
#
# split and embed by sentences
#
#

import chromadb
from nltk.tokenize import sent_tokenize


collection_name = 'all-MiniLM-L6-v2_sentence_split_clean'

persistent_client = chromadb.PersistentClient()

try:
    persistent_client.delete_collection(name=collection_name)
except:
    print('nothing to delete')

collection = persistent_client.get_or_create_collection(collection_name)
display(collection.count())
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


for pdf in pdfs[0:3]:
    with open('txts/' + pdf + '.txt') as f:
        string = f.read()
        cleaner_string = string.replace('\n', ' ').replace('\r', '')

        clean_string = re.sub("\s\s+", " ", cleaner_string)

        # Split text
        split_texts_list = sent_tokenize(clean_string)
        display(len(split_texts_list))

        # Embed text
        embedded_texts = embedding_model.embed_documents(
            texts=split_texts_list)

        # add vectors to collection
        ids = [str(uuid.uuid4()) for sent in split_texts_list]
        metadatas = [{"LPA": pdf}
                     for sent in split_texts_list]
        collection.add(
            embeddings=embedded_texts,
            documents=split_texts_list,
            ids=ids,
            metadatas=metadatas
        )

0

3744

3382

5002